### Plot combined variables against original data 
To run this Notebook follow instructions at https://github.com/mbari-org/auv-python.

In [1]:
import os
import sys
module_path = os.path.abspath(os.path.join('../src/data'));
if module_path not in sys.path:
    sys.path.append(module_path)
import xarray as xr
import hvplot.pandas
import hvplot.xarray
import holoviews as hv
import ipywidgets as widgets
import pylab as plt
import pandas as pd
from pathlib import Path
import netCDF4 as nc4
import textwrap
from nc42netcdfs import BASE_LRAUV_PATH

# Enable bokeh extension for hvplot
hv.extension('bokeh')

# Pick the auv_name
auv_name = widgets.Dropdown(
    options=[f for f in sorted(os.listdir(BASE_LRAUV_PATH)) if f != ".DS_Store"],
    description='auv_name:',
    disabled=False,
)
display(auv_name)

Dropdown(description='auv_name:', options=('ahi', 'brizo', 'daphne', 'data', 'makai', 'pontus', 'tethys', 'tri…

In [2]:
# Pick the log file from the selected LRAUV directory
# Pattern: {lrauv_name}/missionlogs/{year}/{date_range}/{mission_start}/{log_file}.nc4

lrauv_name = auv_name.value
log_files = sorted(Path(BASE_LRAUV_PATH).glob(f"{lrauv_name}/missionlogs/*/*/*/*[0-9].nc4"))
log_file_options = [str(f.relative_to(BASE_LRAUV_PATH)) for f in log_files]

log_file_picker = widgets.Select(
    options=log_file_options,
    description='Log File:',
    disabled=False,
    rows=15,
    layout=widgets.Layout(width='800px')
)
display(log_file_picker)

Select(description='Log File:', layout=Layout(width='800px'), options=('pontus/missionlogs/2024/20240715_20240…

In [7]:
# Read the log_file and the corresponding combined netCDF files into xarray Datasets
log_file = log_file_picker.value
combined_file = log_file.replace('.nc4', '_combined.nc4')

# Open log file with all groups
log_nc = nc4.Dataset(os.path.join(BASE_LRAUV_PATH, log_file))
print("Log file: " + os.path.join(BASE_LRAUV_PATH, log_file))
groups_str = ', '.join(list(log_nc.groups.keys()))
print(f"Groups in log file:\n{textwrap.fill(groups_str, width=80)}\n")
log_ds = xr.open_dataset(os.path.join(BASE_LRAUV_PATH, log_file))
print("Only root group (universals):")
display(log_ds)

print("\nCombined file: " + os.path.join(BASE_LRAUV_PATH, combined_file))
combined_ds = xr.open_dataset(os.path.join(BASE_LRAUV_PATH, combined_file))
display(combined_ds)

Log file: /Users/mccann/GitHub/auv-python/data/lrauv_data/pontus/missionlogs/2024/20240715_20240725/20240723T023501/202407230235_202407232319.nc4
Groups in log file:
controlThread, CycleStarter, CommandExec, NAL9602, Dock, WetLabsUBAT, IBIT,
CBIT, CommandLine, logger, LogSplitter, Vehicle, AHRS_M2, AMEcho,
AcousticModem_Benthos_ATM900, BackseatComponent, BioacousticsDataBridge, BPC1,
BPCW, BR_Ping1D, DataOverHttps, DAT, DATMMP, DDM, DUSBL_Hydroid, Depth_Keller,
DepthKeller33X, DropWeight, DVL_micro, GobyModem, Micromodem, MultiRay, NanoDVR,
Onboard, OnboardBattery, OnboardPressure, Phins_Multibeam, Power24vConverter,
PowerOnly, PNI_TCM, Radio_Surface, RDI_Pathfinder, RDI_PathfinderUp, Rowe_600,
SCPI, Sonardyne_Nano, TrnDataBridge, Waterlinked, Docked, LineCapture, SetNav,
Undock, StratificationFrontDetector, DepAvgTempFrontDetector, BuoyancyServo,
DockingServo, DockingStepper, ElevatorServo, MassServo, RudderServo, ThrusterHE,
ThrusterServo, ExternalSim, ExternalSimGazebo, InternalSim,

<xarray.Dataset> Size: 76MB
Dimensions:                                                              (
                                                                          time_time: 184143,
                                                                          platform_conversation_time: 0,
                                                                          platform_communications_time: 26,
                                                                          depth_time: 184142,
                                                                          platform_pitch_angle_time: 173701,
                                                                          ...
                                                                          concentration_of_colored_dissolved_organic_matter_in_sea_water_time: 0,
                                                                          mission_started_time: 9,
                                                                          eastward_sea_water_velocity_time: 0,
                                                                          northward_sea_water_velocity_time: 0,
                                                                          mole_concentration_of_nitrate_in_sea_water_time: 0,
                                                                          mass_concentration_of_petroleum_hydrocarbons_in_sea_water_time: 0)
Coordinates: (12/76)
    time                                                                 (time_time) float64 1MB ...
  * time_time                                                            (time_time) datetime64[ns] 1MB ...
  * platform_conversation_time                                           (platform_conversation_time) datetime64[ns] 0B ...
  * platform_communications_time                                         (platform_communications_time) datetime64[ns] 208B ...
    depth                                                                (depth_time) float32 737kB ...
  * depth_time                                                           (depth_time) datetime64[ns] 1MB ...
    ...                                                                   ...
  * concentration_of_colored_dissolved_organic_matter_in_sea_water_time  (concentration_of_colored_dissolved_organic_matter_in_sea_water_time) datetime64[ns] 0B ...
  * mission_started_time                                                 (mission_started_time) datetime64[ns] 72B ...
  * eastward_sea_water_velocity_time                                     (eastward_sea_water_velocity_time) datetime64[ns] 0B ...
  * northward_sea_water_velocity_time                                    (northward_sea_water_velocity_time) datetime64[ns] 0B ...
  * mole_concentration_of_nitrate_in_sea_water_time                      (mole_concentration_of_nitrate_in_sea_water_time) datetime64[ns] 0B ...
  * mass_concentration_of_petroleum_hydrocarbons_in_sea_water_time       (mass_concentration_of_petroleum_hydrocarbons_in_sea_water_time) datetime64[ns] 0B ...
Dimensions without coordinates: platform_orientation_matrix_M,
                                platform_orientation_matrix_N, location_fix_M,
                                platform_velocity_wrt_sea_water_M,
                                platform_velocity_wrt_ground_M,
                                acoustic_contact_direction_vehicle_frame_M
Data variables: (12/69)
    trajectory_id                                                        |S15 15B ...
    platform_conversation                                                (platform_conversation_time) uint8 0B ...
    platform_communications                                              (platform_communications_time) uint8 26B ...
    platform_pitch_angle                                                 (platform_pitch_angle_time) float32 695kB ...
    platform_pitch_rate                                                  (platform_pitch_rate_time) float32 736kB ...
    depth_rate          


Combined file: /Users/mccann/GitHub/auv-python/data/lrauv_data/pontus/missionlogs/2024/20240715_20240725/20240723T023501/202407230235_202407232319_combined.nc4


<xarray.Dataset> Size: 70MB
Dimensions:                                                                             (
                                                                                         bpc1_time: 19,
                                                                                         buoyancyservo_time: 3619,
                                                                                         ctdseabird_time: 72810,
                                                                                         currentestimator_time: 49828,
                                                                                         deadreckonusingmultiplevelocitysources_longitude_time: 183919,
                                                                                         ...
                                                                                         wetlabsubat_hv_step_calibration_coefficient_time: 39817,
                                                                                         wetlabsubat_average_bioluminescence_time: 39817,
                                                                                         wetlabsubat_digitized_raw_ad_counts_time: 39817,
                                                                                         digitized_raw_ad_counts_M: 60,
                                                                                         wetlabsubat_digitized_raw_ad_counts_time_60hz: 2389020,
                                                                                         nudged_time: 183919)
Coordinates: (12/32)
  * bpc1_time                                                                           (bpc1_time) datetime64[ns] 152B ...
  * buoyancyservo_time                                                                  (buoyancyservo_time) datetime64[ns] 29kB ...
  * ctdseabird_time                                                                     (ctdseabird_time) datetime64[ns] 582kB ...
  * currentestimator_time                                                               (currentestimator_time) datetime64[ns] 399kB ...
  * deadreckonusingmultiplevelocitysources_longitude_time                               (deadreckonusingmultiplevelocitysources_longitude_time) datetime64[ns] 1MB ...
  * deadreckonusingmultiplevelocitysources_latitude_time                                (deadreckonusingmultiplevelocitysources_latitude_time) datetime64[ns] 1MB ...
    ...                                                                                  ...
  * wetlabsubat_hv_step_calibration_coefficient_time                                    (wetlabsubat_hv_step_calibration_coefficient_time) datetime64[ns] 319kB ...
  * wetlabsubat_average_bioluminescence_time                                            (wetlabsubat_average_bioluminescence_time) datetime64[ns] 319kB ...
  * wetlabsubat_digitized_raw_ad_counts_time                                            (wetlabsubat_digitized_raw_ad_counts_time) datetime64[ns] 319kB ...
  * digitized_raw_ad_counts_M                                                           (digitized_raw_ad_counts_M) int64 480B ...
  * wetlabsubat_digitized_raw_ad_counts_time_60hz                                       (wetlabsubat_digitized_raw_ad_counts_time_60hz) datetime64[ns] 19MB ...
  * nudged_time                                                                         (nudged_time) datetime64[ns] 1MB ...
Data variables: (12/39)
    bpc1_platform_battery_charge                                                        (bpc1_time) float32 76B ...
    bpc1_platform_battery_voltage                                                       (bpc1_time) float32 76B ...
    buoyancyservo_platform_buoyancy_position                                            (buoyancyservo_time) float32 14kB ...
    ctdseabird_sea_water_salinity                                                       (ctdseabird_time) float32 291kB ...
    ctdseabird_sea_

In [8]:
# Select a variable to plot from the combined dataset
variable_picker = widgets.Dropdown(
    options=[var for var in combined_ds.data_vars],
    description='Variable:',
    disabled=False,
)
display(variable_picker)

Dropdown(description='Variable:', options=('bpc1_platform_battery_charge', 'bpc1_platform_battery_voltage', 'b…

In [9]:
# Make a time series plot of the selected variable
variable_name = variable_picker.value
combined_ds[variable_name].hvplot.line(x='ctdseabird_time', title=f'Time Series of {variable_name}', width=1000, height=400)

:Curve   [ctdseabird_time]   (ctdseabird_sea_water_temperature)

In [11]:
# Select a second variable to plot from the combined dataset
variable_picker2 = widgets.Dropdown(
    options=[var for var in combined_ds.data_vars],
    description='Variable:',
    disabled=False,
)
display(variable_picker2)

Dropdown(description='Variable:', options=('bpc1_platform_battery_charge', 'bpc1_platform_battery_voltage', 'b…

In [ ]:
# Create two linked plots (zoom and pan together)
variable_name = variable_picker.value
variable_name2 = variable_picker2.value

# Get time coordinate for each variable by introspection
def get_time_coord(ds, var_name):
    """Get the time coordinate name for a variable."""
    var = ds[var_name]
    # Find dimension that contains 'time' in its name
    time_dims = [dim for dim in var.dims if 'time' in dim.lower()]
    return time_dims[0] if time_dims else var.dims[0]

time_coord1 = get_time_coord(combined_ds, variable_name)
time_coord2 = get_time_coord(combined_ds, variable_name2)

print(f"{variable_name} uses time coordinate: {time_coord1}")
print(f"{variable_name2} uses time coordinate: {time_coord2}")

# Create first plot
plot1 = combined_ds[variable_name].hvplot.line(
    x=time_coord1, 
    title=f'{variable_name}', 
    width=900, 
    height=300
)

# Create second plot linked to first
plot2 = combined_ds[variable_name2].hvplot.line(
    x=time_coord2, 
    title=f'{variable_name2}', 
    width=900, 
    height=300
)

# Stack plots vertically with linked x-axis (time) for synchronized zoom/pan
(plot1 + plot2).cols(1)

BokehModel(combine_events=True, render_bundle={'docs_json': {'e3c5fb25-305b-4c30-82b9-0f8620760499': {'version…

In [ ]:
# Select a second variable to plot
variable_picker2 = widgets.Dropdown(
    options=[var for var in combined_ds.data_vars],
    description='Variable 2:',
    disabled=False,
)
display(variable_picker2)